In [616]:
import pandas as pd
from ggplot import *
import matplotlib.pyplot as plt
%matplotlib inline
import psycopg2
import scipy
import math
import pickle


#Scaling
normilize = True
scale = False

#Spliting features and classifiers
SUBSET_FEATURES = True
SUBMODELS = False

#Showing Each classifier
SEPERATE_CLASSIFIERS = True

#Combining output
NN_COMBINE = True
VOTING_CLASSIFIERS = True


pd.set_option('display.max_columns', None)

try:
    conn = psycopg2.connect("dbname='capstone' user='samkreter' host='localhost'")
except:
    print("I am unable to connect to the database")

cur = conn.cursor()



sql = "select   fights.f1result, \
                f1.height_inches AS f1_height,\
                f1.reach_inches AS f1_reach, \
                f1.weight_lbs AS f1_weight, \
                f1.strike_offense_per_min AS f1_strike_offense_per_min, \
                f1.strike_offense_accuracy AS f1_strike_offense_accuracy, \
                f1.strike_defense_per_min AS f1_strike_defense_per_min, \
                f1.strike_defense_accuracy AS f1_strike_defense_accuracy, \
                f1.takedowns_per_fight AS f1_takedowns_per_fight, \
                f1.takedowns_accuracy AS f1_takedowns_accuracy, \
                f1.takedowns_defense AS f1_takedowns_defense, \
                f1.submissions_per_fight AS f1_submissions_per_fight, \
                f1.total_fights AS f1_total_fights, \
                f1.association AS f1_association, \
                f1.country AS f1_country, \
                f1.wins AS f1_wins, \
                f1.losses AS f1_losses, \
                f1.total_fights AS f1_total_fights, \
                                                \
                f2.height_inches AS f2_height, \
                f2.reach_inches AS f2_reach, \
                f2.weight_lbs AS f2_weight, \
                f2.strike_offense_per_min AS f2_strike_offense_per_min, \
                f2.strike_offense_accuracy AS f2_strike_offense_accuracy, \
                f2.strike_defense_per_min AS f2_strike_defense_per_min, \
                f2.strike_defense_accuracy AS f2_strike_defense_accuracy, \
                f2.takedowns_per_fight AS f2_takedowns_per_fight, \
                f2.takedowns_accuracy AS f2_takedowns_accuracy, \
                f2.takedowns_defense AS f2_takedowns_defense, \
                f2.submissions_per_fight AS f2_submissions_per_fight, \
                f2.total_fights AS f2_total_fights, \
                f2.association AS f2_association, \
                f2.country AS f2_country, \
                f2.wins AS f2_wins, \
                f2.losses AS f2_losses, \
                f2.total_fights AS f2_total_fights \
                from octagon.fights INNER JOIN octagon.fighters AS f1 ON octagon.fights.f1name = f1.name INNER JOIN octagon.fighters AS f2 ON octagon.fights.f2name = f2.name;"

df = pd.read_sql_query(sql, conn)

fighters = pd.read_sql_query("select * from octagon.fighters",conn)



# df = pd.read_sql_query("select * from octagon.fights",conn)
df2 = pd.read_sql_query("select * from octagon.fighters",conn)
conn.close()


In [617]:
test = test.sample(frac=1).reset_index(drop=True)
test.head(n=20)
# print(test['reach_inches'].isnull().sum())
# test = test.interpolate()
# test = test.dropna()
# test.head()
# print(test['reach_inches'].isnull().sum())

,fid,name,dob,height_inches,weight_lbs,association,division,country,reach_inches,strike_offense_per_min,strike_offense_accuracy,strike_defense_per_min,strike_defense_accuracy,takedowns_per_fight,takedowns_accuracy,takedowns_defense,submissions_per_fight,wins,losses,draws,total_fights,win_pct,finish_pct
0,2646,Dan Christison,1972-04-26,80,265,Jackson-Wink MMA,Super Heavyweight,United States,NaN,4.23,51,3.88,54,0.00,0,37,1.1,1,1,0,2,50,100
1,13767,Masanori Kanehara,1982-11-18,67,135,Alpha,Bantamweight,Japan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2,0,3,33,0
2,524,Ronald Jhun,1970-09-21,71,170,808 Top Team,Welterweight,United States,NaN,1.45,57,3.01,50,1.81,100,100,0.0,0,1,0,1,0,0
3,3071,Leigh Remedios,1976-01-13,67,155,Team Quannum,Featherweight,England,NaN,0.75,41,1.36,68,0.00,0,33,0.0,0,1,0,1,0,0
4,124,Don Frye,1965-11-23,73,219,Team Frye,Heavyweight,United States,NaN,2.87,64,3.57,37,0.91,57,45,0.7,0,0,0,0,0,0
5,16984,Tiequan Zhang,1978-07-25,69,155,China Top Team,Featherweight,China,70,1.23,36,2.14,51,1.95,58,75,3.4,1,3,0,4,25,100
6,9033,James Te Huna,1981-09-29,73,185,Pirtek Athletic Allstars,Middleweight,Australia,75,4.04,51,2.99,62,2.75,37,58,0.2,5,4,0,9,56,60
7,1354,Norifumi Yamamoto,1977-03-15,64,135,Krazy Bee,Bantamweight,Japan,66,2.09,45,2.01,57,0.41,22,45,0.0,0,3,0,3,0,0
8,33115,Nazareno Malegarie,1986-05-16,68,155,Team Tavares,Lightweight,Argentina,70,3.87,34,3.27,69,1.00,33,0,0.0,0,1,0,1,0,0
9,15937,Yaotzin Meza,1981-02-04,69,145,MMA Lab,Featherweight,United States,74,1.43,41,1.62,64,1.06,20,18,1.4,2,3,0,5,40,50


### Convert text ones into numeric with normilization

In [601]:


country_names = list(fighters['country'].unique())
df['f1_country'] = df['f1_country'].apply(lambda x: country_names.index(x) / len(country_names))
df['f2_country'] = df['f2_country'].apply(lambda x: country_names.index(x) / len(country_names))


asso_names = list(fighters["association"].unique())
df['f1_association'] = df['f1_association'].apply(lambda x: asso_names.index(x) / len(asso_names))
df['f2_association'] = df['f2_association'].apply(lambda x: asso_names.index(x) / len(asso_names))


# with open("text_based_names.pickle","wb") as f:
#     pickle.dump([country_names,asso_names],f)

# df['f1_association'] = df['f1_association'].apply(lambda x: names.index(x))
# df['f2_association'] = df['f2_association'].apply(lambda x: names.index(x))

df.head()

,f1result,f1_height,f1_reach,f1_weight,f1_strike_offense_per_min,f1_strike_offense_accuracy,f1_strike_defense_per_min,f1_strike_defense_accuracy,f1_takedowns_per_fight,f1_takedowns_accuracy,f1_takedowns_defense,f1_submissions_per_fight,f1_total_fights,f1_association,f1_country,f1_wins,f1_losses,f1_total_fights,f2_height,f2_reach,f2_weight,f2_strike_offense_per_min,f2_strike_offense_accuracy,f2_strike_defense_per_min,f2_strike_defense_accuracy,f2_takedowns_per_fight,f2_takedowns_accuracy,f2_takedowns_defense,f2_submissions_per_fight,f2_total_fights,f2_association,f2_country,f2_wins,f2_losses,f2_total_fights
0,win,78,81,260,2.82,34,2.46,46,3.26,66,50,2.2,4,0.264984,0.000000,1,3,4,79,81,265,0.85,44,2.48,52,2.12,40,33,4.2,3,0.832808,0,1,2,3
1,win,70,NaN,151,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,0.694006,0.000000,4,3,7,70,73,170,1.79,36,2.21,63,2.51,48,64,0.4,25,0.761830,0,15,10,25
2,win,72,NaN,170,1.89,36,3.23,52,0.62,16,58,1.5,3,0.001577,0.000000,1,2,3,68,66,155,2.39,38,2.37,57,2.64,42,86,1.1,3,0.313880,0,1,2,3
3,win,69,74,247,0.97,30,2.87,59,1.06,12,66,0.7,7,0.004732,0.000000,4,3,7,74,NaN,235,2.61,69,3.91,26,4.34,57,66,4.3,2,0.517350,0,0,2,2
4,win,72,73,170,1.61,41,2.00,46,4.17,70,45,0.6,9,0.728707,0.079365,4,5,9,74,NaN,170,2.55,40,2.34,35,0.00,0,0,0.0,1,0.001577,0,0,1,1


In [602]:
df['f1result'] = df['f1result'].replace(['win', 'draw', 'NC'],[1,2,2])
df.head()

,f1result,f1_height,f1_reach,f1_weight,f1_strike_offense_per_min,f1_strike_offense_accuracy,f1_strike_defense_per_min,f1_strike_defense_accuracy,f1_takedowns_per_fight,f1_takedowns_accuracy,f1_takedowns_defense,f1_submissions_per_fight,f1_total_fights,f1_association,f1_country,f1_wins,f1_losses,f1_total_fights,f2_height,f2_reach,f2_weight,f2_strike_offense_per_min,f2_strike_offense_accuracy,f2_strike_defense_per_min,f2_strike_defense_accuracy,f2_takedowns_per_fight,f2_takedowns_accuracy,f2_takedowns_defense,f2_submissions_per_fight,f2_total_fights,f2_association,f2_country,f2_wins,f2_losses,f2_total_fights
0,1,78,81,260,2.82,34,2.46,46,3.26,66,50,2.2,4,0.264984,0.000000,1,3,4,79,81,265,0.85,44,2.48,52,2.12,40,33,4.2,3,0.832808,0,1,2,3
1,1,70,NaN,151,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,0.694006,0.000000,4,3,7,70,73,170,1.79,36,2.21,63,2.51,48,64,0.4,25,0.761830,0,15,10,25
2,1,72,NaN,170,1.89,36,3.23,52,0.62,16,58,1.5,3,0.001577,0.000000,1,2,3,68,66,155,2.39,38,2.37,57,2.64,42,86,1.1,3,0.313880,0,1,2,3
3,1,69,74,247,0.97,30,2.87,59,1.06,12,66,0.7,7,0.004732,0.000000,4,3,7,74,NaN,235,2.61,69,3.91,26,4.34,57,66,4.3,2,0.517350,0,0,2,2
4,1,72,73,170,1.61,41,2.00,46,4.17,70,45,0.6,9,0.728707,0.079365,4,5,9,74,NaN,170,2.55,40,2.34,35,0.00,0,0,0.0,1,0.001577,0,0,1,1


### Subsetting the features to improve performance

In [603]:
if SUBSET_FEATURES:
    subset = ['f1result','f1_strike_offense_per_min','f1_strike_defense_per_min','f1_association','f1_wins','f1_losses', 
        'f2_strike_offense_per_min','f2_strike_defense_per_min','f2_association','f2_wins','f2_losses']
    df = df[subset]

### Interpolate the data to fill in the NAs to get a better result of the data
It was found that dropna works the best for the highest correclation results, We will revisit this later

In [604]:
df0 = df.interpolate()
df0 = df.interpolate(method='spline', order=2)
df = df.interpolate(method='pchip')
df = df.dropna()

df.head()

,f1result,f1_strike_offense_per_min,f1_strike_defense_per_min,f1_association,f1_wins,f1_losses,f2_strike_offense_per_min,f2_strike_defense_per_min,f2_association,f2_wins,f2_losses
0,1,2.820000,2.460000,0.264984,1,3,0.85,2.48,0.832808,1,2
1,1,2.474875,3.065417,0.694006,4,3,1.79,2.21,0.761830,15,10
2,1,1.890000,3.230000,0.001577,1,2,2.39,2.37,0.313880,1,2
3,1,0.970000,2.870000,0.004732,4,3,2.61,3.91,0.517350,0,2
4,1,1.610000,2.000000,0.728707,4,5,2.55,2.34,0.001577,0,1


## Move the lossers to the end to have data for losing people

In [605]:
df.corr()

,f1result,f1_strike_offense_per_min,f1_strike_defense_per_min,f1_association,f1_wins,f1_losses,f2_strike_offense_per_min,f2_strike_defense_per_min,f2_association,f2_wins,f2_losses
f1result,1.000000,-0.035534,-0.026795,-0.017101,-0.001856,-0.025664,-0.016441,-0.025129,0.001472,0.000294,-0.020774
f1_strike_offense_per_min,-0.035534,1.000000,0.358220,-0.019591,0.093729,-0.119533,0.212354,0.175921,0.046782,0.050778,0.024030
f1_strike_defense_per_min,-0.026795,0.358220,1.000000,0.012723,-0.143962,0.019325,0.167807,0.115042,0.005121,-0.072902,-0.029453
f1_association,-0.017101,-0.019591,0.012723,1.000000,-0.142149,-0.123313,0.027255,0.045590,0.008676,-0.079670,-0.075949
f1_wins,-0.001856,0.093729,-0.143962,-0.142149,1.000000,0.539738,-0.002312,-0.095009,-0.039331,0.348939,0.278421
f1_losses,-0.025664,-0.119533,0.019325,-0.123313,0.539738,1.000000,-0.059049,-0.024717,-0.051106,0.167055,0.250639
f2_strike_offense_per_min,-0.016441,0.212354,0.167807,0.027255,-0.002312,-0.059049,1.000000,0.179741,0.035396,0.246571,0.144643
f2_strike_defense_per_min,-0.025129,0.175921,0.115042,0.045590,-0.095009,-0.024717,0.179741,1.000000,-0.051104,-0.187562,-0.089872
f2_association,0.001472,0.046782,0.005121,0.008676,-0.039331,-0.051106,0.035396,-0.051104,1.000000,0.034277,0.010001
f2_wins,0.000294,0.050778,-0.072902,-0.079670,0.348939,0.167055,0.246571,-0.187562,0.034277,1.000000,0.695793


In [606]:
f1_end = math.ceil(len(df.columns) / 2)
df2 = df.copy()
tmp = df2.ix[:,1:f1_end]


df2.ix[:,1:f1_end] = df2.ix[:,f1_end:].values
df2.ix[:,f1_end:] = tmp.values
df2['f1result'] = 0


df2.head()

,f1result,f1_strike_offense_per_min,f1_strike_defense_per_min,f1_association,f1_wins,f1_losses,f2_strike_offense_per_min,f2_strike_defense_per_min,f2_association,f2_wins,f2_losses
0,0,0.85,2.48,0.832808,1,2,2.820000,2.460000,0.264984,1,3
1,0,1.79,2.21,0.761830,15,10,2.474875,3.065417,0.694006,4,3
2,0,2.39,2.37,0.313880,1,2,1.890000,3.230000,0.001577,1,2
3,0,2.61,3.91,0.517350,0,2,0.970000,2.870000,0.004732,4,3
4,0,2.55,2.34,0.001577,0,1,1.610000,2.000000,0.728707,4,5


### Combine the two dataframes and randomize

In [607]:
df = df.append(df2)
df = df.sample(frac=1).reset_index(drop=True)
df.head()

,f1result,f1_strike_offense_per_min,f1_strike_defense_per_min,f1_association,f1_wins,f1_losses,f2_strike_offense_per_min,f2_strike_defense_per_min,f2_association,f2_wins,f2_losses
0,0,2.450000,1.940000,0.111987,7,5,3.09,2.490035,0.004732,17,9
1,1,4.320000,2.940000,0.004732,6,3,1.85,1.860000,0.192429,4,3
2,1,3.382261,2.904815,0.093060,2,2,2.28,2.130000,0.004732,3,2
3,0,4.120000,6.790000,0.184543,0,2,1.74,1.500000,0.115142,3,4
4,0,3.190000,2.610000,0.856467,1,3,3.43,2.470000,0.212934,5,8


In [608]:
#Pull the labels out of the dataset
y = df['f1result']
del df['f1result']

X = df

In [609]:
from sklearn import datasets
# Our Classifier
from sklearn.naive_bayes import GaussianNB
# Get the accuracy score of the final model
from sklearn.metrics import accuracy_score
# Split the training set into train and test
from sklearn.cross_validation import train_test_split
#MLP Classifer
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, VotingClassifier
from sklearn import preprocessing
from sklearn.feature_selection import RFE

if normilize:
    X = preprocessing.normalize(X, norm='l2')
    
if scale:
    X = preprocessing.scale(X)

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=1,random_state = 1)

names = [
         "MLP",
         #"Naive Bayes",
         "KNN",
         "SVM Linear",
         "SVM gamma",
         #"Decsion Tree", 
         #"Random Forest",
         #"adaBoost"
        ]

clfs = [
        MLPClassifier(solver='sgd', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1,max_iter=400),
        #GaussianNB(),
        KNeighborsClassifier(11),
        SVC(kernel="linear", C=0.025,probability=True),
        SVC(gamma=2, C=1,probability=True),
        #DecisionTreeClassifier(max_depth=5),
        #RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
        #AdaBoostClassifier()
       ]

if SUBMODELS and SUBSET_FEATURES:
    #feats = [0,2,3]
    feats = [0,2,3,4,5,6,7]
    clfs = [clfs[i] for i in feats]
    names = [names[i] for i in feats]
elif SUBMODELS:
    feat = [3,5,7]
    clfs = [clfs[i] for i in feats]
    names = [names[i] for i in feats]

#predict_proba to get the probabilities 
    
    #All Features
# SVC- Linear, DTree and AdaBoost are the best especially with Voter
 
#Subset
# MLP, KNN, SVM Linear 
# Normilization Good 
    
    
#13, 15, 16, 30, 
# print("NBC Accuracy of the model is: %.2f%%" % (accuracy_score(nbc_preds,y_test) * 100))
# print("MLP Accuracy of the model is: %.2f%%" % (accuracy_score(mlp_preds,y_test) * 100))


## Finding the importance of each feature using Recursion

In [610]:
if False:
    f1_end = int(len(df.columns) / 2)
    indexs = np.array([i for i in range(f1_end)])
    selector = RFE(clfs[3], 14, step=1)
    selector = selector.fit(X_train, y_train)
    print(selector.support_)
    print(indexs[selector.support_[:f1_end]])
    print(indexs[selector.support_[f1_end:]])
#

## Looking at each Classifier separatly 

In [611]:
if SEPERATE_CLASSIFIERS:
    
    pred_list_train = []
    pred_list_test = []
    
    for index, clf in enumerate(clfs):
        clf.fit(X_train,y_train)

        preds = clf.predict(X_test)
        
        preds_train = np.amax(clf.predict_proba(X_train),axis=1)
        preds_test = np.amax(clf.predict_proba(X_test),axis=1)
        
        pred_list_train.append(preds_train)
        pred_list_test.append(preds_test)
        #print(names[index] + " Accuracy of the model is: %.2f%%" % (accuracy_score(preds,y_test) * 100))
        print(names[index] + ":%.2f" % (accuracy_score(preds,y_test) * 100))

#     with open("clfs-2.pickle","wb") as f:
#         pickle.dump(clfs,f)

MLP:100.00
KNN:100.00
SVM Linear:0.00
SVM gamma:100.00


## Using Voting for Combination

In [612]:
if VOTING_CLASSIFIERS:
    clf_vote = VotingClassifier(estimators=list(zip(names,clfs)),voting='soft')
    clf_vote.fit(X_train,y_train)
    clfs.append(clf_vote)
    
#     with open("clfs-2.pickle","wb") as f:
#         pickle.dump(clfs,f)
        
    preds = clf_vote.predict(X_test)
    print("%.2f" % (accuracy_score(preds,y_test) * 100))


100.00


### Testing Combination of Classifiers using NN - DOES NOT WORK GOOD

In [613]:
if NN_COMBINE:
    combine_clf = MLPClassifier(solver='sgd', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
    combine_clf.fit(np.array(pred_list_train).T,y_train)
    preds = combine_clf.predict(np.array(pred_list_test).T)
    print("%.2f" % (accuracy_score(preds,y_test) * 100))



0.00


In [614]:

try:
    conn = psycopg2.connect("dbname='capstone' user='samkreter' host='localhost'")
except:
    print("I am unable to connect to the database")

cur = conn.cursor()

sql = "select fid, \
            height_inches, \
            reach_inches, \
            weight_lbs, \
            strike_offense_per_min, \
            strike_offense_accuracy, \
            strike_defense_per_min, \
            strike_defense_accuracy, \
            takedowns_per_fight, \
            takedowns_accuracy, \
            takedowns_defense, \
            submissions_per_fight, \
            total_fights, \
            association, \
            country, \
            wins, \
            losses, \
            total_fights FROM octagon.fighters"

#16635
#230

#df1 = pd.read_sql_query(sql + str(16635), conn)
df = pd.read_sql_query("select * from octagon.fighters", conn)

conn.close()


In [615]:
#Figure out good interpolator without having to load the entire database each time
df = df.interpolate()

f1 = df[df['fid'] == 230].drop('fid',axis=1)
f2 = df[df['fid'] == 230].drop('fid',axis=1)

if not f1.empty and not f2.empty:
    print("hello")
#np.append(f1,f2)



hello
